# DVL-NAV -- Unit 770
02/11/2022
Gregory Burgess

---

1. [Import Libraries](#import-libraries)
1. [Load and Parse Data](#load-and-parse-data)
    1. [Bathymetry Data](#import-bathymetry)
    1. [Glider Flight Computer Data](#import-glider-flight-computer)
    1. [Rosbag Data](#import-rosbag-data)
    1. [Separate and Select Dives](#separate-and-select-dives)
    1. [DVL Data](#import-dvl-data)
1. [Compute Water Column Currents](#compute-water-column-currents)
1. [Compute DVL-Odometry](#compute-dvl-odometry)
1. [Multi-Factor Terrain Based Navigation](#multi-factor-terrain-based-navigation)
    1. [Plot Navigation Results](#plot-navigation-results)
    1. [Navigation Performance](#navigation-performance)
        2. [Print Performance metrics](#print_metrics)
1. [Sandbox](#sandbox)


Bin number to 20 --- why did i do this again and do i want 40 for puerto rico

dvl ensemble coordinate frame

---
<a id='import-libraries'></a>
## Import Libraries

In [1]:
import datetime
import earthpy as et
import earthpy.plot as ep
import importlib
import numpy as np
import os
import pandas as pd
import rasterio as rio
import scipy
import scipy.signal
import seaborn as sns 
import struct
import sys
import utm
import unittest
from PIL import Image
from matplotlib import pyplot as plt 
from os import listdir
from os.path import isfile, join
from scipy.spatial.transform import Rotation as R
from scipy import interpolate

# add parent directory to the path for importing modules 
sys.path.insert(1, os.path.join(sys.path[0], '..'))
sys.path.append(os.path.join(sys.path[0], '../data'))

# objects for parsing raw DVL data 
import PathfinderDVL
import PathfinderEnsemble
import PathfinderTimeSeries

# objects for estimating ocean current velocities
import VelocityShearPropagation

# objects for controlling thruster to minimize transport cost 
import AdaptiveVelocityController

# objects for parsing flight and science computer log files
import SlocumFlightController
import SlocumScienceController
import dvl_plotter
import BathymetryMap
import MultiFactorTAN

# data for parsing seafloor bathymetry
import bathy_meta_data
sns.set()


import warnings
warnings.simplefilter('ignore')

def reload_modules():
    importlib.reload(PathfinderDVL)
    importlib.reload(PathfinderEnsemble)
    importlib.reload(PathfinderTimeSeries)
    importlib.reload(VelocityShearPropagation)
    importlib.reload(AdaptiveVelocityController)
    importlib.reload(SlocumFlightController)
    importlib.reload(SlocumScienceController)
    importlib.reload(dvl_plotter)
    importlib.reload(bathy_meta_data)
    importlib.reload(BathymetryMap)
    importlib.reload(MultiFactorTAN)
print('Done!')

#TODO
#pip3 install utm threw error --> is this code all designed for python2???

Done!


In [7]:
filepath = '/home/gburgess/dvl-nav/data/field_data/buzz_bay/dec_15/'

---
<a id='load-and-parse-data'></a>
## Load and Parse Data
<a id='import-bathymetry'></a>
### Bathymetry Data

In [5]:
# #TODO
# reload_modules()
# #Grid_res_num should always be 10 to represent original resolution of the bathymetry chart. Then, the Minimum spatial resolution will be 5m
# #directory = "/mnt/c/Users/User/Dropbox (MIT)/Kolumbo cruise 2019/zduguid/dbd-parsed/sentinel_2019-Nov/
# #10m resolution
# #bathy_df = pd.read_csv('/mnt/c/Users/User/Dropbox (MIT)/Kolumbo cruise 2019/zduguid/bathy/sensitivity_tests/Kolumbo-1.csv')
# bathy_df = pd.read_csv('/home/gburgess/dvl-nav/data/field_data/kolumbo/bathy/Kolumbo-1.csv')
# grid_res_num = 10
# # map_var_resolution = '10m'

# #20m resolution
# #bathy_df_var = pd.read_csv('/mnt/c/Users/User/Dropbox (MIT)/Kolumbo cruise 2019/zduguid/bathy/sensitivity_tests/Kolumbo-2.csv')
# bathy_df_var = pd.read_csv('/home/gburgess/dvl-nav/data/field_data/kolumbo/bathy/Kolumbo-2.csv')
# map_var_resolution = '20m'
# # grid_res_num = 20

<a id='import-glider-flight-computer'></a>
### Glider Flight Computer Data

In [8]:
reload_modules()
directory = filepath+'dbd_parsed/'
ts_flight = SlocumFlightController.SlocumFlightController.from_directory(directory, save=False, verbose=False)
#ts_flight.df.to_csv(filepath+'ts_flight.csv', sep=',' , index=False)

>> Parsing folder of ASC Files
>> Finished Parsing!


<a id='import-rosbag-data'></a>
### Rosbag Data

In [12]:
# CTD Sensor
ctd_df = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'ctd-data.csv')

# IMU Sensor
ahrs_df = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'devices-spartonm2-ahrs.csv')

# DVL Sensor
dvl_df        = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'devices-dvl-dvl.csv')
dvl_pd0_df    = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'devices-dvl-pd0.csv')
dvl_ranges_df = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'devices-dvl-ranges.csv')
dvl_raw_df    = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'devices-dvl-instrument-raw.csv')

# EXTCTL - GLider 
m_lon_df     = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'extctl-sensors-m_lon.csv')
m_lat_df     = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'extctl-sensors-m_lat.csv')
m_gps_lon_df    = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'extctl-sensors-m_gps_lon.csv')
m_gps_lat_df    = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'extctl-sensors-m_gps_lat.csv')
m_depth_df      = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'extctl-sensors-m_depth.csv')
m_pitch_df      = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'extctl-sensors-m_pitch.csv')
m_roll_df       = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'extctl-sensors-m_roll.csv')
m_altitude_df   = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'extctl-sensors-m_altitude.csv')
m_alt_status_df = pd.read_csv(filepath+ 'rosbags/rosbag_output/' + 'extctl-sensors-m_altimeter_status.csv')

# Micron Sonar



<a id='separate-and-select-dives'></a>
### Separate and Select Dives

In [ ]:
#TODO

<a id='import-dvl-data'></a>
### DVL Data

In [ ]:
#TODO

---
<a id='compute-water-column-currents'></a>
## Compute Water Column Currents


In [ ]:
#TODO

---
<a id='compute-dvl-odometry'></a>
## Compute DVL-Odometry

In [2]:
#TODO

---
## Plot Navigation Results 

In [4]:
#TODO